In [ ]:
# 필수 라이브러리 불러오기 

import numpy as np
from math import sqrt, pi

from qiskit import *
from qiskit.visualization import plot_histogram, plot_distribution, plot_error_map
from qiskit.visualization.array import array_to_latex
from qiskit.quantum_info import Operator, Statevector, random_statevector

from qiskit_ibm_provider import IBMProvider

from qiskit_aer import AerSimulator, StatevectorSimulator

import matplotlib.pyplot as plt

In [ ]:
#IBMProvider.save_account("Your_Token", overwrite=True)
#provider = IBMProvider()

#수업을 위해 추가된 허브와 그룹, 프로젝트를 프로바이더에 설정
provider = IBMProvider(instance="ibm-q-education/ibm-quantum-1/hallimu")

In [ ]:
provider.backends() #5 5Q 7Q Backends

수업을 위해 제공되는 백엔드들 중 가장 적은 Queue를 지닌 백엔드를 찾아 실습을 위한 백엔드로 설정합니다. 

In [ ]:
from qiskit_ibm_provider import least_busy

real_backend = least_busy(provider.backends())
real_backend

### 새로운 양자 게이트 - RY

#### 표준 회전 게이트들

표준 회전 게이트들은 블로흐 구면의 $X, Y, Z$축을 기준으로 하는 회전 연산자들로 정의됩니다.

#### RY 게이트

RY 게이트는 Y축을 중심으로 큐비트의 상태를 주어진 $\theta$만큼 회전하는 연산자 입니다. 

$$
R_y(\theta) =
\begin{pmatrix}
\cos(\theta/2) & - \sin(\theta/2)\\
\sin(\theta/2) & \cos(\theta/2)
\end{pmatrix})
$$

In [ ]:
ry_gate = QuantumCircuit(1)
ry_gate.ry(np.pi/2,0)
ry_gate.draw(output='mpl')

In [ ]:
Statevector(ry_gate).draw('bloch')

## 불량배의 대장이 여러분에게 보여 준 것
(contents from: [The Quantum Enigmas 004 - The Monty Hall problem](https://www.youtube.com/watch?v=Hd9KhRts1uw) of Institut quantique de l'Université de Sherbrooke)


불량배의 대장 앞에는 세개의 금고가 있습니다.

<img src="img/enigma2.png" alt="Alternative text" width=800/>

그리고 여러분께 이야기 합니다.

너의 가방과 책을 열 수 있는 열쇠는 이 금고중 하나에 숨겨져 있다. 네가 한번에 찾으면 모두 돌려 줄 것이고 찾지 못하면 모두 영원히 나의 것이다.

여러분은 고민끝에 2번을 선택했습니다. 불량배의 대장은 남은 두 개 중 하나의 금고인 1번 금고의 문을 열어서 당신에게 보여줍니다. 열쇠와 가방이 들어있지 않네요!

이제 불량배 대장은 당신에게 질문합니다.

"선택을 바꿀 기회를 한번 주겠다. 너의 선택을 바꾸겠느냐?"

여러분의 선택은 무엇입니까?


### 1단계, 가방과 열쇠가 숨겨진 금고의 상태를 만들어 보자.

가방과 열쇠는 세개의 금고중 하나에 숨겨져 있습니다. 0번째 금고에 가방과 열쇠가 숨겨져 있는 상태를 $|001\rangle$, 1번째 금고에 가방과 열쇠가 숨겨져 있는 상태를 $|010\rangle$, 2번째 금고에 가방과 열쇠가 숨겨져 있는 상태를 $|100\rangle$이라 한다면 양자 회로의 초기 상태는 이 세가지 상태가 모두 같은 확률로 존재하는 상태일 것입니다. 
$$|\psi_0\rangle  = \frac{1}{\sqrt{3}}(|001\rangle + |010\rangle + |100\rangle)$$

가장 먼저, 0번째 큐비트의 상태가 1로 측정될 확률이 1/3, 0으로 측정될 확률이 2/3이 되도록 회로를 만들어 봅시다.

이 초기 상태를 만드는 방법을 고민해 봅시다.

In [ ]:
#your method




저는 이 상태를 구현하기 위해 회로를 원하는 상태로 초기화한 후, transpile옵션을 지정해서 초기화된 양자상태가 어떤 회로로 구성되는지 확인해보려 합니다./


In [ ]:
#방법  transpiler 이용

qc = QuantumCircuit(3)
init = [0, sqrt(1/3) ,sqrt(1/3), 0 , sqrt(1/3), 0, 0 , 0 ]
qc.initialize(init)
target_basis = ['ry', 'cx', 'h']
qc_init = transpile(qc,basis_gates=target_basis)
qc_init.draw('mpl')

In [ ]:
plot_histogram(Statevector(qc_init).sample_counts(90000))

이렇게 만들어진 회로를 사용해서 문제를 이어 나갑시다.

In [ ]:
stage2 = QuantumCircuit(3,1)
stage2.append(qc_init, [0,1,2])
Statevector(stage2).draw('bloch')

In [ ]:
plot_histogram(Statevector(stage2).sample_counts(90000))

이제 문제의 상황을 단계별로 생각해봅시다. 세 금고의 상태는 서로 얽혀 있습니다. 그리고 이중 하나에만 가방과 열쇠가 담겨져 있습니다. 
불량배 두목의 요청으로 여러분은 이미 금고 2번에 구슬이 있을것이라 예측했습니다. 

이때 가정할 수 있는 상황은 두가지 입니다. 2번에 있을 경우와, 그렇지 않은 경우입니다. 어느 경우이든, 불량배의 두목은 여러분이 선택하지 않은 금고 중 하나의 문을 열어 여러분에게 보여주었습니다. 문제의 경우 1번 금고였습니다.

불량배의 두목이 문을 열어서 보여준 상황을 생각해 봅시다. 불량배 두목이 선택하는 금고의 번호를 저장할 새로운 큐비트를 하나 추가합시다.

In [ ]:
select_q = QuantumRegister(1)
stage2.add_bits(select_q)
stage2.draw('mpl')

select_q의 초기 상태는 0이며 이것은 0번 금고를 불량배의 두목이 연 경우를 의미합니다. 불량배의 두목이 0번 또는 1번 금고의 문을 열어서 보여준 상황을 각각의 금고에 가방과 열쇠가 있는 경우를 감안하여 만들어 봅시다.

In [ ]:
# your code

stage2.draw('mpl')

In [ ]:
plot_histogram(Statevector(stage2).sample_counts(90000))

이 경우, 두목은 1번 금고를 열게 되며, 두목이 열어서 보여주는 금고의 번호는 1이 됩니다. 

이제 2번 금고에 가방과 열쇠가 있는 경우를 생각해 봅시다. 이 경우 두목이 열어서 보여줄 수 있는 금고는 0 또는 1이 되며, 이것은 Controlled hadamard 게이트를 사용해 간단히 구현될 수 있습니다.

In [ ]:
#your code

stage2.draw('mpl')

In [ ]:
plot_histogram(Statevector(stage2).sample_counts(90000))

이 결과를 분석해 봅시다. 불량배 두목은 여러분에게 1번 금고를 열어서 보여주었습니다. 이 경우 발생할 수 있는 경우의 수는 0번 금고에 가방과 열쇠가 있을 경우와 2번 금고에 가방과 열쇠가 있는 경우인데, 놀랍게도 0번 금고에 가방과 열쇠가 있을 확률이 2배 더 높습니다.

따라서 정답은, 선택을 바꾸는 것이 가방과 열쇠를 찾을 확률을 높인다. 가 됩니다. 

# 축하합니다! 여러분은 선택을 바꾸어 관문을 통과했습니다. 이제 열쇠를 사용해 책을 열어 봅시다.

In [ ]:
plot_error_map(real_backend)


In [ ]:
stage2.measure_all()
stage2.draw('mpl')

In [ ]:
qc_tr = transpile(stage2, real_backend)
real_job = real_backend.run(qc_tr, shots = 20000)
job_id = real_job.job_id()
print(job_id)

In [ ]:
ret_job = provider.retrieve_job(job_id)
ret_job.status()

In [ ]:
real_counts = ret_job.result().get_counts()
plot_histogram(real_counts)